# Lab | Pandas

In this lab, we will be working with the customer data from an insurance company, which can be found in the CSV file located at the following link: https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

The data includes information such as customer ID, state, gender, education, income, and other variables that can be used to perform various analyses.

Throughout the lab, we will be using the pandas library in Python to manipulate and analyze the data. Pandas is a powerful library that provides various data manipulation and analysis tools, including the ability to load and manipulate data from a variety of sources, including CSV files.

### Data Description

- Customer - Customer ID

- ST - State where customers live

- Gender - Gender of the customer

- Education - Background education of customers 

- Customer Lifetime Value - Customer lifetime value(CLV) is the total revenue the client will derive from their entire relationship with a customer. In other words, is the predicted or calculated value of a customer over their entire duration as a policyholder with the insurance company. It is an estimation of the net profit that the insurance company expects to generate from a customer throughout their relationship with the company. Customer Lifetime Value takes into account factors such as the duration of the customer's policy, premium payments, claim history, renewal likelihood, and potential additional services or products the customer may purchase. It helps insurers assess the long-term profitability and value associated with retaining a particular customer.

- Income - Customers income

- Monthly Premium Auto - Amount of money the customer pays on a monthly basis as a premium for their auto insurance coverage. It represents the recurring cost that the insured person must pay to maintain their insurance policy and receive coverage for potential damages, accidents, or other covered events related to their vehicle.

- Number of Open Complaints - Number of complaints the customer opened

- Policy Type - There are three type of policies in car insurance (Corporate Auto, Personal Auto, and Special Auto)

- Vehicle Class - Type of vehicle classes that customers have Two-Door Car, Four-Door Car SUV, Luxury SUV, Sports Car, and Luxury Car

- Total Claim Amount - the sum of all claims made by the customer. It represents the total monetary value of all approved claims for incidents such as accidents, theft, vandalism, or other covered events.


External Resources: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

## Challenge 1: Understanding the data

In this challenge, you will use pandas to explore a given dataset. Your task is to gain a deep understanding of the data by analyzing its characteristics, dimensions, and statistical properties.

- Identify the dimensions of the dataset by determining the number of rows and columns it contains.
- Determine the data types of each column and evaluate whether they are appropriate for the nature of the variable. You should also provide suggestions for fixing any incorrect data types.
- Identify the number of unique values for each column and determine which columns appear to be categorical. You should also describe the unique values of each categorical column and the range of values for numerical columns, and give your insights.
- Compute summary statistics such as mean, median, mode, standard deviation, and quartiles to understand the central tendency and distribution of the data for numerical columns. You should also provide your conclusions based on these summary statistics.
- Compute summary statistics for categorical columns and providing your conclusions based on these statistics.

In [1]:
import pandas as pd
url = 'http://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv'
insurance_df = pd.read_csv(url)

#1.A number of rows and columns library
insurance_df.shape

#1.B Determine the data types of each column and evaluate whether they are appropriate for the nature of the variable. You should also provide suggestions for fixing any incorrect data types. data cleaning
insurance_df.dtypes

#The list includes many rows that are full of NAN, hence we limit the lenght of the list to the rows with values
insurance_df_clean = insurance_df.dropna(how='all')

#Columns that should have a different data type are: Customer Lifetime Value converted to number and Number of Open Complaints with sum of all values separated by '/'. Reason: Customer Lifetime Value represents a monetary value and should be numeric for calculations, we transform it to numbersand get rid of the %. Number of Open Complaints should be an integer representing the total count of complaints, we sum all the complaints separated by '/' to get a total number.
insurance_df_clean['Customer Lifetime Value'] = insurance_df_clean['Customer Lifetime Value'].str.rstrip('%').astype(float) / 100

insurance_df_clean['Customer Lifetime Value'] = pd.to_numeric(
    insurance_df_clean['Customer Lifetime Value'], errors='coerce'
)
insurance_df_clean['Number of Open Complaints'] = insurance_df_clean['Number of Open Complaints'].apply(
    lambda x: sum([int(num) for num in str(x).split('/') if num.isdigit()]))

#1C - Identify the number of unique values for each column and determine which columns appear to be categorical.
print("Unique values per column:")
print(insurance_df_clean.nunique())
#The categorical columns appear to be: ST, GENDER, Policy Type, Vehicle Class and they have few unique values as string types.

#Describe the unique values of each categorical column and the range of values for numerical columns, and give your insights.
#ST = is the state, it has 51 unique values, representing different states in the US
#Gender, should have a limited number of option based on the region where the data was collected, it has a minimum of 2 unique values, the list includes different ways to write genders, capitals, lower case initials, full words ... it has now been standarised.
gender_map = {
    'Femal': 'F',
    'female': 'F',
    'Male': 'M'
}
insurance_df_clean['GENDER'] = insurance_df_clean['GENDER'].replace(gender_map)

print(insurance_df_clean['GENDER'].dropna().unique()[:3])

# 1.D. Summary statistics for numerical columns
# List of numerical columns
numerical_columns = ['Customer Lifetime Value', 'Income', 'Monthly Premium Auto', 'Number of Open Complaints', 'Total Claim Amount']

# Dictionary to store results
summary_stats = {}

for col in numerical_columns:
    col_data = insurance_df_clean[col].dropna()
    summary_stats[col] = {
        'mean': col_data.mean(),
        'median': col_data.median(),
        'mode': col_data.mode().iloc[0] if not col_data.mode().empty else np.nan,
        'std_dev': col_data.std(),
        '25%': col_data.quantile(0.25),
        '50%': col_data.quantile(0.50),
        '75%': col_data.quantile(0.75)
    }

# Display results
for col, stats_dict in summary_stats.items():
    print(f"\nSummary statistics for '{col}':")
    for stat_name, value in stats_dict.items():
        print(f"{stat_name.capitalize()}: {value:.2f}")

# Basic conclusions
print("\nConclusions:")
for col, stats_dict in summary_stats.items():
    mean = stats_dict['mean']
    median = stats_dict['median']
    std_dev = stats_dict['std_dev']
    print(f"- '{col}': Mean = {mean:.2f}, Median = {median:.2f}, Std Dev = {std_dev:.2f}")
    if abs(mean - median) > std_dev * 0.5:
        print(f"  → The distribution of '{col}' appears to be skewed.")
    else:
        print(f"  → The distribution of '{col}' appears to be relatively symmetric.")

# After looking at the summary stats, a few things stand out. Some columns like Customer Lifetime Value and Income have a wide range and are clearly influenced by a few very high values. That means the average might not tell the full story, and the median is probably more useful in those cases.
# Monthly Premium Auto has some extreme values that don’t look right—one of them is way too high—so it’s worth checking for errors or outliers there.
# Number of Open Complaints doesn’t vary much at all. Most people have just one, so it might make more sense to treat it as a category instead of a number.
# Total Claim Amount looks more balanced, but still has some variation. 
# Overall, the data could use a bit of cleaning—fixing outliers, standardising values, and maybe rethinking how we treat some columns before doing deeper analysis or modelling.

# 1.C.  Numerical columns &  the number of unique values for each numerical column
numerical_columns = [col for col in insurance_df_clean.columns if insurance_df_clean[col].dtype in ['int64', 'float64']]

print("\nNumerical columns and their value ranges:")
for col in numerical_columns:
    min_val = insurance_df_clean[col].min()
    max_val = insurance_df_clean[col].max()
    unique_vals = insurance_df_clean[col].nunique()
    print(f"{col}: min={min_val}, max={max_val}, unique values={unique_vals}")

# identify and describe the categorical columns
categorical_columns = [col for col in insurance_df_clean.columns if insurance_df_clean[col].dtype == 'object' or insurance_df_clean[col].nunique() < 20]

print("\nCategorical columns and their unique values:")
for col in categorical_columns:
    unique_vals = insurance_df_clean[col].dropna().unique()
    print(f"{col}: {unique_vals}")

# Insights: There is a wide range in data, from customer ages 18 to 95, and income $10K to $250K. Standarisation was required for Gender, however it has several missing entries, talking about missing entries customer lifetime value also has many missing values that could impact analysis. Categorical columns like Policy Type and Vehicle Class have limited unique values, making them suitable for categorical analysis. Overall, the dataset appears to be a mix of numerical and categorical data, with some cleaning needed for accurate analysis. There is also inconsistency in the way some categorical data is recorded, which may require further cleaning like State. California vs Cali, or Education Bachelors vs Bachelor. In addition, Monthly premium Auto has some extreme values that may be a mistake, if the highes slary is $250K, the monthly value of iver $34K seems not aligned with the rest of the data.

# 1.D. Summary statistics for numerical columns
# List of numerical columns
numerical_columns = ['Customer Lifetime Value', 'Income', 'Monthly Premium Auto', 'Number of Open Complaints', 'Total Claim Amount']

# Dictionary to store results
summary_stats = {}

for col in numerical_columns:
    col_data = insurance_df_clean[col].dropna()
    summary_stats[col] = {
        'mean': col_data.mean(),
        'median': col_data.median(),
        'mode': col_data.mode().iloc[0] if not col_data.mode().empty else np.nan,
        'std_dev': col_data.std(),
        '25%': col_data.quantile(0.25),
        '50%': col_data.quantile(0.50),
        '75%': col_data.quantile(0.75)
    }

# Display results
for col, stats_dict in summary_stats.items():
    print(f"\nSummary statistics for '{col}':")
    for stat_name, value in stats_dict.items():
        print(f"{stat_name.capitalize()}: {value:.2f}")

# Basic conclusions
print("\nConclusions:")
for col, stats_dict in summary_stats.items():
    mean = stats_dict['mean']
    median = stats_dict['median']
    std_dev = stats_dict['std_dev']
    print(f"- '{col}': Mean = {mean:.2f}, Median = {median:.2f}, Std Dev = {std_dev:.2f}")
    if abs(mean - median) > std_dev * 0.5:
        print(f"  → The distribution of '{col}' appears to be skewed.")
    else:
        print(f"  → The distribution of '{col}' appears to be relatively symmetric.")

# After looking at the summary stats, a few things stand out. Some columns like Customer Lifetime Value and Income have a wide range and are clearly influenced by a few very high values. That means the average might not tell the full story, and the median is probably more useful in those cases.
# Monthly Premium Auto has some extreme values that don’t look right—one of them is way too high—so it’s worth checking for errors or outliers there.
# Number of Open Complaints doesn’t vary much at all. Most people have just one, so it might make more sense to treat it as a category instead of a number.
# Total Claim Amount looks more balanced, but still has some variation. 
# Overall, the data could use a bit of cleaning—fixing outliers, standardising values, and maybe rethinking how we treat some columns before doing deeper analysis or modelling.


Unique values per column:
Customer                     1071
ST                              8
GENDER                          5
Education                       6
Customer Lifetime Value      1027
Income                        774
Monthly Premium Auto          132
Number of Open Complaints       6
Policy Type                     3
Vehicle Class                   6
Total Claim Amount            761
dtype: int64
['F' 'M']

Summary statistics for 'Customer Lifetime Value':
Mean: 7936.90
Median: 5881.74
Mode: 2514.59
Std_dev: 6434.78
25%: 4034.08
50%: 5881.74
75%: 8962.87

Summary statistics for 'Income':
Mean: 39295.70
Median: 36234.00
Mode: 0.00
Std_dev: 30469.43
25%: 14072.00
50%: 36234.00
75%: 64631.00

Summary statistics for 'Monthly Premium Auto':
Mean: 193.23
Median: 83.00
Mode: 65.00
Std_dev: 1601.19
25%: 68.00
50%: 83.00
75%: 109.50

Summary statistics for 'Number of Open Complaints':
Mean: 1.39
Median: 1.00
Mode: 1.00
Std_dev: 0.88
25%: 1.00
50%: 1.00
75%: 1.00

Summary statistics

/var/folders/mg/wjsd2dzj6zgdzk_gsdz19rdc0000gn/T/ipykernel_18212/770837903.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insurance_df_clean['Customer Lifetime Value'] = insurance_df_clean['Customer Lifetime Value'].str.rstrip('%').astype(float) / 100
/var/folders/mg/wjsd2dzj6zgdzk_gsdz19rdc0000gn/T/ipykernel_18212/770837903.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insurance_df_clean['Customer Lifetime Value'] = pd.to_numeric(
/var/folders/mg/wjsd2dzj6zgdzk_gsdz19rdc0000gn/T/ipykernel_1821

## Challenge 2: analyzing the data

### Exercise 1

The marketing team wants to know the top 5 less common customer locations. Create a pandas Series object that contains the customer locations and their frequencies, and then retrieve the top 5 less common locations in ascending order.

In [2]:
# Count how many times each location appears
location_counts = insurance_df_clean['ST'].value_counts()

# Get the 5 locations that appear the least
least_common = location_counts.nsmallest(5)

# Show the result
print(least_common)

ST
AZ             25
WA             30
Washington     81
Nevada         98
Cali          120
Name: count, dtype: int64


### Exercise 2

The sales team wants to know the total number of policies sold for each type of policy. Create a pandas Series object that contains the policy types and their total number of policies sold, and then retrieve the policy type with the highest number of policies sold.

*Hint:*
- *Using value_counts() method simplifies this analysis.*
- *Futhermore, there is a method that returns the index of the maximum value in a column or row.*


In [3]:
# Count how many times each policy type appears
policy_counts = insurance_df_clean['Policy Type'].value_counts()

# Show the total policies sold for each type
print("Total policies sold by type:")
print(policy_counts)

# Find the policy type with the highest number
most_sold = policy_counts.idxmax()

print("\nPolicy type with the highest number of policies sold:")
print(most_sold)

Total policies sold by type:
Policy Type
Personal Auto     780
Corporate Auto    234
Special Auto       57
Name: count, dtype: int64

Policy type with the highest number of policies sold:
Personal Auto


### Exercise 3

The sales team wants to know if customers with Personal Auto have a lower income than those with Corporate Auto. How does the average income compare between the two policy types?

- Use *loc* to create two dataframes: one containing only Personal Auto policies and one containing only Corporate Auto policies.
- Calculate the average income for each policy.
- Print the results.

In [4]:
# Create two DataFrames using .loc
personal_auto = insurance_df_clean.loc[insurance_df_clean['Policy Type'] == 'Personal Auto']
corporate_auto = insurance_df_clean.loc[insurance_df_clean['Policy Type'] == 'Corporate Auto']

# Calculate average income for each group
avg_income_personal = personal_auto['Income'].mean()
avg_income_corporate = corporate_auto['Income'].mean()

# Print the results
print("Average income for Personal Auto customers:", round(avg_income_personal, 2))
print("Average income for Corporate Auto customers:", round(avg_income_corporate, 2))

Average income for Personal Auto customers: 38180.7
Average income for Corporate Auto customers: 41390.31


### Bonus: Exercise 4


Your goal is to identify customers with a high policy claim amount.

Instructions:

- Review again the statistics for total claim amount to gain an understanding of the data.
- To identify potential areas for improving customer retention and profitability, we want to focus on customers with a high policy claim amount. Consider customers with a high policy claim amount to be those in the top 25% of the total claim amount. Create a pandas DataFrame object that contains information about customers with a policy claim amount greater than the 75th percentile.
- Use DataFrame methods to calculate summary statistics about the high policy claim amount data. 

*Note: When analyzing data, we often want to focus on certain groups of values to gain insights. Percentiles are a useful tool to help us define these groups. A percentile is a measure that tells us what percentage of values in a dataset are below a certain value. For example, the 75th percentile represents the value below which 75% of the data falls. Similarly, the 25th percentile represents the value below which 25% of the data falls. When we talk about the top 25%, we are referring to the values that fall above the 75th percentile, which represent the top quarter of the data. On the other hand, when we talk about the bottom 25%, we are referring to the values that fall below the 25th percentile, which represent the bottom quarter of the data. By focusing on these groups, we can identify patterns and trends that may be useful for making decisions and taking action.*

*Hint: look for a method that gives you the percentile or quantile 0.75 and 0.25 for a Pandas Series.*

*Hint 2: check `Boolean selection according to the values of a single column` in https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9*

In [5]:
# 1: Calculate the 75th percentile for claims and complaints
claim_75 = insurance_df_clean['Total Claim Amount'].quantile(0.75)
complaints_75 = insurance_df_clean['Number of Open Complaints'].quantile(0.75)

# 2: Filter customers with high claims and high complaints
high_risk_customers = insurance_df_clean[
    (insurance_df_clean['Total Claim Amount'] > claim_75) &
    (insurance_df_clean['Number of Open Complaints'] > complaints_75)
]

# 3: Show summary statistics, rounded to 2 decimal places
summary = high_risk_customers.describe().round(2)

# 4: Print the results
print("75th percentile - Total Claim Amount:", round(claim_75, 2))
print("75th percentile - Number of Open Complaints:", round(complaints_75, 2))
print("\nSummary statistics for customers with high claims and high complaints:")
print(summary)

75th percentile - Total Claim Amount: 532.8
75th percentile - Number of Open Complaints: 1.0

Summary statistics for customers with high claims and high complaints:
       Customer Lifetime Value   Income  Monthly Premium Auto  \
count                    58.00     58.0                 58.00   
mean                   9190.30  23389.5                135.91   
std                    6466.17  26279.9                122.36   
min                    2450.19      0.0                 63.00   
25%                    5228.91      0.0                 99.75   
50%                    8059.16  20332.5                113.50   
75%                    9989.69  38213.5                127.25   
max                   38445.86  99002.0               1005.00   

       Number of Open Complaints  Total Claim Amount  
count                      58.00               58.00  
mean                        2.90              779.18  
std                         1.12              368.97  
min                         2

In [6]:
#preview
insurance_df.head(50)

,Customer,ST,GENDER,Education,Customer Lifetime Value,Income,Monthly Premium Auto,Number of Open Complaints,Policy Type,Vehicle Class,Total Claim Amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
5,OC83172,Oregon,F,Bachelor,825629.78%,62902.0,69.0,1/0/00,Personal Auto,Two-Door Car,159.383042
6,XZ87318,Oregon,F,College,538089.86%,55350.0,67.0,1/0/00,Corporate Auto,Four-Door Car,321.600000
7,CF85061,Arizona,M,Master,721610.03%,0.0,101.0,1/0/00,Corporate Auto,Four-Door Car,363.029680
8,DY87989,Oregon,M,Bachelor,2412750.40%,14072.0,71.0,1/0/00,Corporate Auto,Four-Door Car,511.200000
9,BQ94931,Oregon,F,College,738817.81%,28812.0,93.0,1/0/00,Special Auto,Four-Door Car,425.527834


In [7]:
#preview
insurance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4008 entries, 0 to 4007
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Customer                   1071 non-null   object 
 1   ST                         1071 non-null   object 
 2   GENDER                     954 non-null    object 
 3   Education                  1071 non-null   object 
 4   Customer Lifetime Value    1068 non-null   object 
 5   Income                     1071 non-null   float64
 6   Monthly Premium Auto       1071 non-null   float64
 7   Number of Open Complaints  1071 non-null   object 
 8   Policy Type                1071 non-null   object 
 9   Vehicle Class              1071 non-null   object 
 10  Total Claim Amount         1071 non-null   float64
dtypes: float64(3), object(8)
memory usage: 344.6+ KB
